In [1]:
import os
from os.path import join
from io import StringIO
from textwrap import dedent

import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model, err_dicke_single

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [2]:
qnet.init_printing()

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-06-25_mg_dicke1_QSL_nonherm.json',
}
register_doit_as_IPython_magic()

In [4]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [5]:
! qdyn_version

QDYN 2.0dev revision 07208674cba8010244458bc99c1f99ffe948bba0 (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=openmpi, parallel-oct, backtraces, no-debug
  compiled with ifort on Sun Jun 25 21:05:52 2017 on host mlhpc2


The data in `./data/dicke1_QSL_nonherm/` that this notebook generates continues partially from `./data/dicke1_QSL_1traj_xxx/`, which was generated from a buggy version of QDYN that should be *very* close to to a proper non-Hermitian optimization.

We use `n_nodes=2, T=2` as a test-case to repeat an entire optimization.

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# QSL for Single-Excitation Dicke state

## action wrappers

In [6]:
DATA_ROOT = './data/dicke1_QSL_nonherm'

In [7]:
def update_config(rf, lambda_a, iter_stop):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [8]:
def submit_optimization(rf, n_trajs, task):
    """Asynchronously run optimization"""
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    assert n_trajs == 1
    body = dedent(r'''
    {module_load}

    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS=1 {runner} qdyn_optimiz --J_T=J_T_sm .
    ''')
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    runner = {
        'copper': 'aprun -B',
        'mlhpc': 'mpirun -n %d' % n_trajs}
    if jobscript.cluster_name != 'copper':
        jobscript.ppn = int(n_trajs)
        if n_trajs == 1:
            runner['mlhpc'] = ''
    jobscript.runner = runner[jobscript.cluster_name]
    jobscript.n_trajs = str(int(n_trajs))
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))

In [9]:
def wait_for_clusterjob(dumpfile):
    """Wait until the clusterjob.AsyncResult cached in the given dumpfile ends"""
    try:
        run = clusterjob.AsyncResult.load(dumpfile)
        run.wait()
        os.unlink(dumpfile)
        return run.successful()
    except OSError:
        pass

In [11]:
def rewrite_to_nonherm(rf):
    """Rewrite the config file in `rf` that was originally written for
    an MCWF optimization into a config file for a non-Hermitian
    optimization"""
    changed = False
    config_data = QDYN.config.read_config_file(join(rf, 'config'))
    if 'dissipator' in config_data:
        del config_data['dissipator']
        changed = True
    if config_data['prop'].get('use_mcwf', False):
        config_data['prop']['use_mcwf'] = False
        changed = True
        if 'mcwf_order' in config_data['prop']:
            del config_data['prop']['mcwf_order']
    if changed:
        QDYN.config.write_config(config_data, join(rf, 'config'))

## custom uptodate routines

In [12]:
from src.qdyn_model_v1 import pulses_uptodate

## task definitions

In [13]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    if frac_part == '00':
        rf = 'rf_%dnodes_T%s' % (row['n_nodes'], int_part)
    else:
        rf = 'rf_%dnodes_T%s_%s' % (row['n_nodes'], int_part, frac_part)
    return join(DATA_ROOT, rf)

In [14]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if n_nodes not in slh_models:
            slh_mode
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=False, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']))),
                (write_dicke_single_model, [slh, ], dict(
                    rf=join(rf, 'prop_traj'), T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=True, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))
            ],
            'targets': [join(rf, 'config'), join(rf, 'prop_traj', 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [15]:
def task_rewrite_to_nonherm():
    """Rewrite config for use of non-Hermitian Hamiltonian
    
    This task is used (manually, one-time) to convert
    runfolders copied from ./data/dicke1_QSL_1traj_xxx/
    """
    jobs = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (rewrite_to_nonherm, [rf, ])],
            'task_dep': ['create_runfolder:%s' % rf],
            'uptodate': [False, ] # up to date if target exists
        }
    for job in jobs.values():
        yield job

In [16]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [17]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ], dict(n_trajs=row['n_trajs']))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [18]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

##  OCT Submission

In [19]:
params_data_str = r'''
# n_nodes   T     lambda_a  n_trajs   iter_stop
        2   0.50     0.001        1        1000
        2   0.50    0.0005        1       15000
        2   1.00     0.001        1        1000
        2   1.00    0.0005        1       30000
        2   1.10     0.001        1        1000
        2   1.10    0.0005        1       30000
        2   1.20     0.001        1        1000
        2   1.20    0.0005        1       30000
        2   1.25     0.001        1        1000
        2   1.25    0.0005        1       30000
        2   1.30     0.001        1        1000
        2   1.30    0.0005        1       30000
        2   1.40     0.001        1        1000
        2   1.40    0.0005        1       30000
        2   1.50     0.001        1        1000
        2   1.50    0.0005        1       30000
        2   1.60     0.001        1        1000
        2   1.60    0.0005        1       30000
        2   1.75     0.001        1        1000
        2   1.75    0.0005        1       15000
        2   1.75    0.0005        1       30000
        2   1.80     0.001        1        1000
        2   1.80    0.0005        1       30000
        2   1.90     0.001        1        1000
        2   1.90    0.0005        1       30000
        2   2.00     0.001        1        1000
        2   2.00    0.0005        1       30000
        2   2.10     0.001        1        1000
        2   2.10    0.0005        1       30000
        2   2.20     0.001        1        1000
        2   2.20    0.0005        1       30000
        2   2.25     0.001        1        1000
        2   2.25    0.0005        1       15000
        2   2.30     0.001        1        1000
        2   2.30    0.0005        1       30000
        2   2.50     0.001        1        1000
        2   2.50    0.0005        1       20000
        2   3.00     0.001        1        1000
        2   3.00    0.0005        1       20000
        2   4.00     0.001        1        1000
        2   4.00    0.0005        1       20000
        2   5.00     0.001        1        1000
        2   5.00     0.001        1       20000
        2   7.00     0.001        1        1000
        2   7.00     0.001        1       20000
        2  10.00     0.001        1        1000
        2  10.00    0.0005        1        2000
        2  20.00     0.001        1        1000
        2  50.00     0.001        1        1000
        2  50.00    0.0005        1        2000
        2  70.00     0.001        1        1000
        2  70.00    0.0005        1        2000
        3   0.50     0.001        1        1000
        3   0.50    0.0005        1       20000
        3   1.00     0.001        1        1000
        3   1.00    0.0005        1       20000
        3   1.25     0.001        1        1000
        3   1.25    0.0005        1       30000
        3   1.30     0.001        1        1000
        3   1.30    0.0005        1       30000
        3   1.40     0.001        1        1000
        3   1.40    0.0005        1       30000
        3   1.50     0.001        1        1000
        3   1.50    0.0005        1       30000
        3   1.60     0.001        1        1000
        3   1.60    0.0005        1       30000
        3   1.70     0.001        1        1000
        3   1.70    0.0005        1       30000
        3   1.75     0.001        1        1000
        3   1.75    0.0005        1       15000
        3   1.80     0.001        1        1000
        3   1.80    0.0005        1       30000
        3   1.90     0.001        1        1000
        3   1.90    0.0005        1       30000
        3   2.00     0.001        1        1000
        3   2.00    0.0005        1       30000
        3   2.10     0.001        1        1000
        3   2.10    0.0005        1       30000
        3   2.20     0.001        1        1000
        3   2.20    0.0005        1       30000
        3   2.25     0.001        1        1000
        3   2.25    0.0005        1       15000
        3   2.30     0.001        1        1000
        3   2.30    0.0005        1       30000
        3   2.40     0.001        1        1000
        3   2.40    0.0005        1       30000
        3   2.50     0.001        1        1000
        3   2.50    0.0005        1       15000
        3   3.00     0.001        1        1000
        3   3.00    0.0005        1       10000
        3   4.00     0.001        1        1000
        3   4.00    0.0005        1       10000
        3   5.00     0.001        1        1000
        3   5.00    0.0005        1       10000
        3   8.00     0.001        1        1000
        3   8.00    0.0005        1       10000
        3   9.00     0.001        1        1000
        3   9.00    0.0005        1        2000
        3  10.00     0.001        1        1000
        3  10.00    0.0005        1        2000
        3  15.00     0.001        1        1000
        3  20.00     0.001        1        1000
        3  50.00     0.001        1        1000
        3  50.00    0.0005        1        2000
        3  70.00     0.001        1        1000
        3  70.00    0.0005        1        2000
        4   0.50     0.001        1        1000
        4   0.50    0.0005        1       20000
        4   0.60     0.001        1        1000
        4   0.60    0.0005        1       20000
        4   0.75     0.001        1        1000
        4   0.75    0.0005        1       30000
        4   0.80     0.001        1        1000
        4   0.80    0.0005        1       30000
        4   0.90     0.001        1        1000
        4   0.90    0.0005        1       20000
        4   1.00     0.001        1        1000
        4   1.00    0.0005        1       10000
        4   1.00    0.0005        1       20000
        4   1.25     0.001        1        1000
        4   1.25    0.0005        1       20000
        4   1.50     0.001        1        1000
        4   1.50    0.0005        1       20000
        4   1.60     0.001        1        1000
        4   1.60    0.0005        1       20000
        4   1.70     0.001        1        1000
        4   1.70    0.0005        1       20000
        4   1.75     0.001        1        1000
        4   1.75    0.0005        1       20000
        4   1.80     0.001        1        1000
        4   1.80    0.0005        1       20000
        4   1.90     0.001        1        1000
        4   1.90    0.0005        1       20000
        4   2.00     0.001        1        1000
        4   2.00    0.0005        1       20000
        4   2.20     0.001        1        1000
        4   2.20    0.0005        1       20000
        4   2.50     0.001        1        1000
        4   2.50    0.0005        1       20000
        4   3.00     0.001        1        1000
        4   3.00    0.0005        1       20000
        4   4.00     0.001        1        1000
        4   4.00    0.0005        1       20000
        4   5.00     0.001        1        1000
        4   5.00    0.0005        1       20000
        4   6.00     0.001        1        1000
        4   6.00    0.0005        1       20000
        4   7.00     0.001        1        1000
        4   7.00    0.0005        1       20000
        4   8.00     0.001        1        1000
        4   8.00    0.0005        1       15000
        4   9.00     0.001        1        1000
        4   9.00    0.0005        1        4000
        4  10.00     0.001        1        1000
        4  10.00    0.0005        1        4000
        4  15.00     0.001        1        1000
        4  20.00     0.001        1        1000
        4  50.00     0.001        1        1000
        4  70.00     0.001        1        1000
        5   0.50     0.001        1        1000
        5   0.50    0.0005        1       20000
        5   0.75     0.001        1        1000
        5   0.75    0.0005        1       20000
        5   1.00     0.001        1        1000
        5   1.00    0.0005        1       20000
        5   1.25     0.001        1        1000
        5   1.25    0.0005        1       20000
        5   1.50     0.001        1        1000
        5   1.50    0.0005        1       20000
        5   1.75     0.001        1        1000
        5   1.75    0.0005        1       20000
        5   2.00     0.001        1        1000
        5   2.00    0.0005        1       20000
        5   2.10    0.0005        1       10000
        5   2.10    0.0005        1       20000
        5   2.20    0.0005        1       10000
        5   2.20    0.0005        1       20000
        5   2.30    0.0005        1       10000
        5   2.30    0.0005        1       15000
        5   2.40    0.0005        1       10000
        5   2.40    0.0005        1       20000
        5   2.50     0.001        1        1000
        5   2.50    0.0005        1       15000
        5   2.60     0.001        1        1000
        5   2.60    0.0005        1       15000
        5   2.75     0.001        1        1000
        5   2.75    0.0005        1       15000
        5   2.80     0.001        1        1000
        5   2.80    0.0005        1       15000
        5   2.90     0.001        1        1000
        5   2.90    0.0005        1       15000
        5   3.00     0.001        1        1000
        5   3.00    0.0005        1       20000
        5   4.00     0.001        1        1000
        5   4.00    0.0005        1       20000
        5   5.00     0.001        1        1000
        5   5.00    0.0005        1       20000
        5   6.00     0.001        1        1000
        5   6.00    0.0005        1       20000
        5   7.00     0.001        1        1000
        5   7.00    0.0005        1       20000
        5   8.00     0.001        1        1000
        5   8.00    0.0005        1       20000
        5   9.00     0.001        1        1000
        5   9.00    0.0005        1       10000
        5   9.00    0.0005        1       20000
        5  10.00     0.001        1        1000
        5  10.00    0.0005        1        8000
        5  12.00     0.001        1        1000
        5  12.00    0.0005        1        2000
        5  15.00     0.001        1        1000
        5  15.00    0.0005        1        2000
        5  18.00     0.001        1        1000
        5  18.00    0.0005        1        2000
        5  20.00     0.001        1        1000
        5  20.00    0.0005        1        2000
        5  50.00     0.001        1        1000
        5  50.00    0.0005        1        2000
        5  70.00     0.001        1        1000
        5  70.00    0.0005        1        2000
        6   0.50     0.001        1        1000
        6   0.50    0.0005        1       10000
        6   0.60     0.001        1        1000
        6   0.60    0.0005        1       10000
        6   0.75     0.001        1        1000
        6   0.75    0.0005        1       20000
        6   0.80     0.001        1        1000
        6   0.80    0.0005        1       10000
        6   0.90     0.001        1        1000
        6   0.90    0.0005        1       10000
        6   1.00     0.001        1        1000
        6   1.00    0.0005        1       20000
        6   1.25     0.001        1        1000
        6   1.25    0.0005        1       20000
        6   1.50     0.001        1        1000
        6   1.50    0.0005        1       20000
        6   1.75     0.001        1        1000
        6   1.75    0.0005        1       20000
        6   2.00     0.001        1        1000
        6   2.00    0.0005        1       20000
        6   2.50     0.001        1        1000
        6   2.50    0.0005        1       20000
        6   3.00     0.001        1        1000
        6   3.00    0.0005        1       20000
        6   3.10     0.001        1        1000
        6   3.10    0.0005        1       20000
        6   3.20     0.001        1        1000
        6   3.20    0.0005        1       20000
        6   3.25     0.001        1        1000
        6   3.25    0.0005        1       20000
        6   3.30     0.001        1        1000
        6   3.30    0.0005        1       20000
        6   3.40     0.001        1        1000
        6   3.40    0.0005        1       20000
        6   3.50     0.001        1        1000
        6   3.50    0.0005        1       20000
        6   3.60     0.001        1        1000
        6   3.60    0.0005        1       20000
        6   3.70     0.001        1        1000
        6   3.70    0.0005        1       20000
        6   3.80     0.001        1        1000
        6   3.80    0.0005        1       20000
        6   3.90     0.001        1        1000
        6   3.90    0.0005        1       20000
        6   4.00     0.001        1        1000
        6   4.00    0.0005        1       20000
        6   4.25     0.001        1        1000
        6   4.25    0.0005        1       20000
        6   4.50     0.001        1        1000
        6   4.50    0.0005        1       20000
        6   5.00     0.001        1        1000
        6   5.00    0.0005        1       20000
        6   5.50     0.001        1        1000
        6   5.50    0.0005        1       20000
        6   6.00    0.0005        1       10000
        6   6.00    0.0005        1       20000
        6   7.00    0.0005        1       10000
        6   7.00    0.0005        1       20000
        6   8.00     0.001        1        1000
        6   8.00    0.0005        1       20000
        6   8.00    0.0005        1       15000
        6   9.00     0.001        1        1000
        6   9.00    0.0005        1       10000
        6   9.00    0.0005        1       15000
        6  10.00     0.001        1        1000
        6  10.00    0.0005        1       10000
        6  12.00     0.001        1        1000
        6  12.00    0.0005        1       10000
        6  14.00     0.001        1        1000
        6  14.00    0.0005        1        5000
        6  14.00    0.0005        1       10000
        6  15.00     0.001        1        1000
        6  15.00    0.0005        1        2000
        6  18.00     0.001        1        1000
        6  18.00    0.0005        1        2000
        6  20.00     0.001        1        1000
        6  20.00    0.0005        1        2000
        6  50.00     0.001        1        1000
        6  50.00    0.0005        1        2000
        6  70.00     0.001        1        1000
        6  70.00    0.0005        1        2000
        7   1.00     0.001        1        1000
        7   1.00    0.0005        1       20000
        7   1.50     0.001        1        1000
        7   1.50    0.0005        1       20000
        7   1.75     0.001        1        1000
        7   1.75    0.0005        1       20000
        7   2.00     0.001        1        1000
        7   2.00    0.0005        1       20000
        7   2.50     0.001        1        1000
        7   2.50    0.0005        1       20000
        7   3.00     0.001        1        1000
        7   3.00    0.0005        1       20000
        7   3.50     0.001        1        1000
        7   3.50    0.0005        1       20000
        7   4.00     0.001        1        1000
        7   4.00    0.0005        1       20000
        7   4.50     0.001        1        1000
        7   4.50    0.0005        1       20000
        7   5.00     0.001        1        1000
        7   5.00    0.0005        1       20000
        7   5.50     0.001        1        1000
        7   5.50    0.0005        1       20000
        7   6.00     0.001        1        1000
        7   6.00    0.0005        1       20000
        7   7.00     0.001        1        1000
        7   7.00    0.0005        1       20000
        7   8.00     0.001        1        1000
        7   8.00    0.0005        1       20000
        7   9.00     0.001        1        1000
        7   9.00    0.0005        1       20000
        7  10.00     0.001        1        1000
        7  10.00    0.0005        1       20000
        7  12.00     0.001        1        1000
        7  12.00    0.0005        1       10000
        7  14.00     0.001        1        1000
        7  14.00    0.0005        1        5000
        7  14.00    0.0005        1       10000
        7  15.00     0.001        1        1000
        7  15.00    0.0005        1        2000
        8  10.00     0.001        1        1000
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'lambda_a', 'n_trajs', 'iter_stop'],
        converters={'n_nodes': int, 'T': float, 'lambda_a': float,
                    'n_trajs': int, 'iter_stop': int})

In [20]:
print(params_df.to_string())

     n_nodes      T  lambda_a  n_trajs  iter_stop
0          2   0.50    0.0010        1       1000
1          2   0.50    0.0005        1      15000
2          2   1.00    0.0010        1       1000
3          2   1.00    0.0005        1      30000
4          2   1.10    0.0010        1       1000
5          2   1.10    0.0005        1      30000
6          2   1.20    0.0010        1       1000
7          2   1.20    0.0005        1      30000
8          2   1.25    0.0010        1       1000
9          2   1.25    0.0005        1      30000
10         2   1.30    0.0010        1       1000
11         2   1.30    0.0005        1      30000
12         2   1.40    0.0010        1       1000
13         2   1.40    0.0005        1      30000
14         2   1.50    0.0010        1       1000
15         2   1.50    0.0005        1      30000
16         2   1.60    0.0010        1       1000
17         2   1.60    0.0005        1      30000
18         2   1.75    0.0010        1       1000


In [21]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, filename='./data/dicke1_nonherm_QSL_clusterjob.log')

In [22]:
#%doit -n 20 rewrite_to_nonherm

In [ ]:
%doit -n 20 wait_for_optimization

-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T0_50
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_10
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_20
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_25
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_30
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_40
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_50
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_60
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_75
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_80
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T1_90
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T2
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T2_10
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T2_20
-- create_runfolder:./data/dicke1_QSL_nonherm/rf_2nodes_T2_25
-- create_runf

.  update_runfolder:0
.  update_runfolder:2
.  update_runfolder:4
.  update_runfolder:6
.  update_runfolder:8
.  update_runfolder:10
.  update_runfolder:12
.  update_runfolder:16
.  update_runfolder:14
.  update_runfolder:18
.  update_runfolder:23
.  update_runfolder:21
.  update_runfolder:25
.  update_runfolder:27
.  update_runfolder:29
.  update_runfolder:31
.  update_runfolder:33
.  update_runfolder:35
.  update_runfolder:37
.  update_runfolder:39
.  update_runfolder:54
.  update_runfolder:43
.  update_runfolder:45
.  update_runfolder:50
.  update_runfolder:41
.  update_runfolder:52
.  update_runfolder:47
.  update_runfolder:48
.  update_runfolder:56
.  update_runfolder:60
.  update_runfolder:66
.  update_runfolder:78
.  update_runfolder:74
.  update_runfolder:64
.  update_runfolder:72
.  update_runfolder:68
.  update_runfolder:76
.  update_runfolder:62
.  update_runfolder:90
.  update_runfolder:70
.  update_runfolder:58
.  update_runfolder:104
.  update_runfolder:84
.  update_runfo

-- submit_optimization:279
-- submit_optimization:285
-- submit_optimization:281
-- submit_optimization:293
-- submit_optimization:283
-- submit_optimization:305
-- submit_optimization:289
-- submit_optimization:295
-- submit_optimization:299
-- submit_optimization:309
-- submit_optimization:297
-- submit_optimization:277
-- submit_optimization:291
-- submit_optimization:307
-- submit_optimization:311
-- submit_optimization:303
-- submit_optimization:314
.  wait_for_optimization:271
.  wait_for_optimization:273
.  wait_for_optimization:275
.  wait_for_optimization:287
.  submit_optimization:301
.  update_runfolder:5
.  update_runfolder:3
.  submit_optimization:316
.  update_runfolder:11
.  update_runfolder:1
.  update_runfolder:7
.  update_runfolder:9
.  update_runfolder:26
.  update_runfolder:28
.  update_runfolder:30
.  update_runfolder:32
.  update_runfolder:65
.  update_runfolder:75
.  update_runfolder:15
.  update_runfolder:22
.  update_runfolder:73
.  update_runfolder:17
.  updat

-- submit_optimization:105
-- submit_optimization:87
.  wait_for_optimization:89
-- submit_optimization:55
.  wait_for_optimization:77
-- submit_optimization:42
-- submit_optimization:36
.  wait_for_optimization:114
.  wait_for_optimization:116
.  wait_for_optimization:95
-- submit_optimization:53
-- submit_optimization:276
.  wait_for_optimization:107
-- submit_optimization:142
.  wait_for_optimization:112
.  wait_for_optimization:99
.  wait_for_optimization:124
-- submit_optimization:83
-- submit_optimization:103
.  wait_for_optimization:91
-- submit_optimization:272
.  wait_for_optimization:120
.  wait_for_optimization:118
-- submit_optimization:274
-- submit_optimization:136
.  wait_for_optimization:109
.  wait_for_optimization:128
.  wait_for_optimization:126
.  wait_for_optimization:81
.  wait_for_optimization:101
.  wait_for_optimization:97
.  wait_for_optimization:122
.  wait_for_optimization:130
.  wait_for_optimization:154
.  wait_for_optimization:170
.  wait_for_optimization